# L2: Filtering With Metadata

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>


In [ ]:
# 警告控制
import warnings

# 忽略所有警告信息，以免干扰程序输出
warnings.filterwarnings('ignore')


In [ ]:
import custom_utils  # 导入自定义工具库 custom_utils

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils</code> files:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Data Loading

In [ ]:
# 1. 加载数据集
from datasets import load_dataset
import pandas as pd

# 加载数据集
dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split="train")
dataset = dataset.take(100)  # 取前100条数据

# 将数据集转换为 pandas 数据框
dataset_df = pd.DataFrame(dataset)

# 显示前5条数据
dataset_df.head(5)

In [ ]:
# 打印数据框的列名
print("Columns:", dataset_df.columns)

## Document Modelling

In [ ]:
# 使用自定义工具库处理记录，并将结果存储在 listings 变量中
listings = custom_utils.process_records(dataset_df)

## Database Creation and Connection

In [ ]:
# 使用自定义工具库连接到数据库，并获取数据库和集合对象
db, collection = custom_utils.connect_to_database()

In [ ]:
# 删除集合中所有现有的记录
collection.delete_many({})

## Data Ingestion

In [ ]:
# 数据插入过程可能需要几分钟时间
collection.insert_many(listings)
print("Data ingestion into MongoDB completed")

## Vector Search Index defintion

In [ ]:
# 使用自定义工具库设置向量搜索索引
custom_utils.setup_vector_search_index(collection=collection)

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note:</b> If the output of the previous cell is <code>Error creating vector search index: Duplicate Index</code> you may proceed to the next cell if you intend to still use a previously created index.</p>

## Compose Vector Search Query

In [ ]:
def vector_search(user_query, db, collection, additional_stages=[], vector_index="vector_index_text"):
    """
    在 MongoDB 集合中基于用户查询执行向量搜索。

    Args:
    user_query (str): 用户的查询字符串。
    db (MongoClient.database): 数据库对象。
    collection (MongoCollection): 要搜索的 MongoDB 集合。
    additional_stages (list): 额外的聚合阶段要包括在管道中。
    vector_index (str): 向量索引名称，默认为 "vector_index_text"。

    Returns:
    list: 匹配的文档列表。
    """

    # 为用户查询生成嵌入向量
    query_embedding = custom_utils.get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # 定义向量搜索阶段
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,  # 指定用于搜索的索引
            "queryVector": query_embedding,  # 表示查询的向量
            "path": "text_embeddings",  # 文档中包含要搜索向量的字段
            "numCandidates": 150,  # 考虑的候选匹配数
            "limit": 20,  # 返回前20个匹配结果
        }
    }

    # 定义包含向量搜索阶段和其他阶段的聚合管道
    pipeline = [vector_search_stage] + additional_stages

    # 执行搜索
    results = collection.aggregate(pipeline)

    # 解释查询执行计划
    explain_query_execution = db.command(
        'explain', {  # 返回有关 MongoDB 如何执行查询或命令的信息，而无需实际运行它
            'aggregate': collection.name,  # 指定执行聚合的集合名称
            'pipeline': pipeline,  # 要分析的聚合管道
            'cursor': {}  # 指示应使用默认游标行为
        }, 
        verbosity='executionStats'  # 有关聚合管道每个阶段执行的详细统计信息
    )

    # 获取向量搜索的解释信息
    vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
    millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

    # 打印数据库服务器上执行完成所需的总时间
    print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")

    return list(results)  # 返回搜索结果列表

## Handling User Query

In [ ]:
from pydantic import BaseModel
from typing import Optional
import custom_utils

class SearchResultItem(BaseModel):
    name: str  # 房源名称
    accommodates: Optional[int] = None  # 可容纳人数，可选
    bedrooms: Optional[int] = None  # 卧室数量，可选
    address: custom_utils.Address  # 地址信息
    space: str = None  # 空间描述，可选

In [ ]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    """
    处理用户查询并返回系统响应和源信息。

    Args:
    query (str): 用户的查询字符串。
    db (MongoClient.database): 数据库对象。
    collection (MongoCollection): 要搜索的 MongoDB 集合。
    stages (list): 额外的聚合阶段要包括在管道中。
    vector_index (str): 向量索引名称，默认为 "vector_index_text"。

    Returns:
    str: 系统响应。
    """
    # 执行向量搜索
    get_knowledge = vector_search(query, db, collection, stages, vector_index)

    # 检查是否有结果
    if not get_knowledge:
        return "No results found.", "No source information available."

    # 将搜索结果转换为 SearchResultItem 模型列表
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # 将搜索结果转换为 DataFrame 以便在 Jupyter 中更好地呈现
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # 使用 OpenAI 的 completion 生成系统响应
    completion = custom_utils.openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "You are an Airbnb listing recommendation system."
            },
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )

    system_response = completion.choices[0].message['content']

    # 打印用户问题、系统响应和源信息
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # 以 HTML 表格形式显示 DataFrame
    display(HTML(search_results_df.to_html()))

    # 返回结构化响应和源信息作为字符串
    return system_response

## Adding A Post Filter to Vector Search (Match Operator)

In [ ]:
import re

# 指定要限制文档的元数据字段
search_path = "address.country"

# 创建匹配阶段
match_stage = {
    "$match": {
        search_path: re.compile(r"United States"),  # 匹配国家为美国的文档
        "accommodates": { "$gt": 1, "$lt": 5 }  # 匹配可容纳人数在1到5之间的文档
    }
}

# 额外的聚合阶段
additional_stages = [match_stage]

In [ ]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from restaurants, can you recommend a place? 
Include a reason as to why you've chosen your selection.
"""

# 处理用户查询并获取响应
response = handle_user_query(query, db, collection, additional_stages)
response

## Adding A PreFilter to Vector Search

In [ ]:
from pymongo.operations import SearchIndexModel
import time

# 新向量索引名称
vector_index_with_filter = "vector_index_with_filter"

# 定义新的向量搜索索引模型
new_vector_search_index_model = SearchIndexModel(
    definition={
        "mappings": {
            "dynamic": True,
            "fields": {
                "text_embeddings": {
                    "dimensions": 1536,
                    "similarity": "cosine",
                    "type": "knnVector",
                },
                "accommodates": {
                    "type": "number"
                },
                "bedrooms": {
                    "type": "number"
                },
            },
        }
    },
    name=vector_index_with_filter,
)

# 创建新索引
try:
    result = collection.create_search_index(model=new_vector_search_index_model)
    print("Creating index...")
    time.sleep(20)  # 休眠20秒，确保向量索引在使用前完成初始同步
    print("New index created successfully:", result)
except Exception as e:
    print(f"Error creating new vector search index: {str(e)}")

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note:</b> If the output of the previous cell is <code>Error creating vector search index: Duplicate Index</code> you may proceed to the next cell if you intend to still use a previously created index.</p>

In [ ]:
def vector_search(user_query, db, collection, additional_stages=[], vector_index="vector_index_text"):
    """
    在 MongoDB 集合中基于用户查询执行向量搜索。

    Args:
    user_query (str): 用户的查询字符串。
    db (MongoClient.database): 数据库对象。
    collection (MongoCollection): 要搜索的 MongoDB 集合。
    additional_stages (list): 额外的聚合阶段要包括在管道中。
    vector_index (str): 向量索引名称，默认为 "vector_index_text"。

    Returns:
    list: 匹配的文档列表。
    """

    # 为用户查询生成嵌入向量
    query_embedding = custom_utils.get_embedding(user_query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # 定义向量搜索阶段
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,  # 指定用于搜索的索引
            "queryVector": query_embedding,  # 表示查询的向量
            "path": "text_embeddings",  # 文档中包含要搜索向量的字段
            "numCandidates": 150,  # 考虑的候选匹配数
            "limit": 20,  # 返回前20个匹配结果
            "filter": {
                "$and": [
                    {"accommodates": {"$gte": 2}}, 
                    {"bedrooms": {"$lte": 7}}
                ]
            },
        }
    }
    # 定义包含向量搜索阶段和其他阶段的聚合管道
    pipeline = [vector_search_stage] + additional_stages

    # 执行搜索
    results = collection.aggregate(pipeline)

    # 解释查询执行计划
    explain_query_execution = db.command(
        'explain', {  # 返回有关 MongoDB 如何执行查询或命令的信息，而无需实际运行它
            'aggregate': collection.name,  # 指定执行聚合的集合名称
            'pipeline': pipeline,  # 要分析的聚合管道
            'cursor': {}  # 指示应使用默认游标行为
        }, 
        verbosity='executionStats'  # 有关聚合管道每个阶段执行的详细统计信息
    )

    # 获取向量搜索的解释信息
    vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
    millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

    # 打印数据库服务器上执行完成所需的总时间
    print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")
    
    return list(results)  # 返回搜索结果列表

In [ ]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from restaurants, can you recommend a place? 
Include a reason as to why you've chosen your selection.
"""

# 处理用户查询并获取响应
response = handle_user_query(
    query, 
    db, 
    collection, 
    vector_index=vector_index_with_filter
)
response